<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC1_Turbofan_FD003_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.svm import LinearSVR

from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.exceptions import ConvergenceWarning

from sklearn.feature_selection import SelectFromModel
from sklearn.tree import DecisionTreeRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [3]:
sns.set_palette('colorblind')

# Data Preparation

In [4]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [5]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [6]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [7]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(3)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0005,0.0004,100.0,518.67,642.36,1583.23,1396.84,14.62,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,1,2,0.0008,-0.0003,100.0,518.67,642.50,1584.69,1396.89,14.62,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,1,3,-0.0014,-0.0002,100.0,518.67,642.18,1582.35,1405.61,14.62,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,1,4,-0.0020,0.0001,100.0,518.67,642.92,1585.61,1392.27,14.62,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,1,5,0.0016,0.0000,100.0,518.67,641.68,1588.63,1397.65,14.62,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24715,100,148,-0.0016,-0.0003,100.0,518.67,643.78,1596.01,1424.11,14.62,...,519.66,2388.30,8138.08,8.5036,0.03,394,2388,100.0,38.44,22.9631
24716,100,149,0.0034,-0.0003,100.0,518.67,643.29,1596.38,1429.14,14.62,...,519.91,2388.28,8144.36,8.5174,0.03,395,2388,100.0,38.50,22.9746
24717,100,150,-0.0016,0.0004,100.0,518.67,643.84,1604.53,1431.41,14.62,...,519.44,2388.24,8135.95,8.5223,0.03,396,2388,100.0,38.39,23.0682
24718,100,151,-0.0023,0.0004,100.0,518.67,643.94,1597.56,1426.57,14.62,...,520.01,2388.26,8141.24,8.5148,0.03,395,2388,100.0,38.31,23.0753


## Preprocessing

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [8]:
test.shape, y_test.shape

((16596, 26), (100, 1))

In [9]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.59,1592.40,1409.87,14.62,21.58,560.53,2388.22,9085.50,1.31,...,528.05,2388.23,8158.77,8.2966,0.03,393,2388,100.0,39.43,23.5679
1,518.67,642.56,1587.42,1409.69,14.62,21.61,553.33,2388.18,9050.97,1.30,...,520.90,2388.17,8128.04,8.4514,0.03,392,2388,100.0,38.83,23.2821
2,518.67,642.75,1591.93,1417.66,14.62,21.60,563.61,2388.31,9091.69,1.31,...,531.36,2388.33,8173.56,8.3057,0.03,395,2388,100.0,39.27,23.6440
3,518.67,642.28,1584.68,1406.56,14.62,21.61,552.75,2388.07,9048.23,1.30,...,521.27,2388.09,8133.78,8.4337,0.03,392,2388,100.0,38.70,23.3804
4,518.67,642.15,1580.59,1397.26,14.62,21.58,553.82,2387.96,9050.89,1.30,...,521.74,2387.96,8132.51,8.3900,0.03,390,2388,100.0,38.89,23.4463


In [10]:
X_test = test_last

### Remaining Useful Life (RUL)

In [11]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,258
1,1,2,257
2,1,3,256
3,1,4,255
4,1,5,254


## Attributes and target separation

In [12]:
X_train, y_train = proc.X_y_train_divide(train)

In [13]:
y_train.head()

,RUL
0,258
1,257
2,256
3,255
4,254


In [14]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.36,1583.23,1396.84,14.62,21.61,553.97,2387.96,9062.17,1.3,...,522.31,2388.01,8145.32,8.4246,0.03,391,2388,100.0,39.11,23.3537
1,518.67,642.50,1584.69,1396.89,14.62,21.61,554.55,2388.00,9061.78,1.3,...,522.42,2388.03,8152.85,8.4403,0.03,392,2388,100.0,38.99,23.4491
2,518.67,642.18,1582.35,1405.61,14.62,21.61,554.43,2388.03,9070.23,1.3,...,522.03,2388.00,8150.17,8.3901,0.03,391,2388,100.0,38.85,23.3669
3,518.67,642.92,1585.61,1392.27,14.62,21.61,555.21,2388.00,9064.57,1.3,...,522.49,2388.08,8146.56,8.3878,0.03,392,2388,100.0,38.96,23.2951
4,518.67,641.68,1588.63,1397.65,14.62,21.61,554.74,2388.04,9076.14,1.3,...,522.58,2388.03,8147.80,8.3869,0.03,392,2388,100.0,39.14,23.4583


## Training and Evaluation functions

In [15]:
eval = Evaluation()

In [16]:
search = HyperparameterSearch()

# LinearRegression

## Linear RUL

In [17]:
model = LinearRegression()

In [18]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.587,RMSE=-63.494
(CV) R2=0.543,RMSE=-65.175


In [19]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.900,RMSE=-57.057


## Non-linear RUL

In [20]:
y_train.clip(upper=189).head()

,RUL
0,189
1,189
2,189
3,189
4,189


In [21]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [23]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 121}}
Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f4790d58c20>,
                                    

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 121}}
```



In [24]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':121})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f4790d58c20>,
                                                                            kw_args={'a_max': 121,
                                                                                     'a_min': 0})))])

In [25]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.776,RMSE=-18.503
(CV) R2=0.762,RMSE=-18.871


In [26]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.720,RMSE=-20.287


Just adding a non-linear improved results, as in the first set

## PolyFeatures + Linear RUL

In [27]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('lin_reg'  ,   LinearRegression())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('lin_reg', LinearRegression())])

In [28]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 3
n_required_iterations: 3
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 12
n_resources: 500
Fitting 5 folds for each of 12 candidates, totalling 60 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:296: UserWarning: The total space of parameters 12 is smaller than n_iter=49. Running 12 iterations. For exhaustive searches, use GridSearchCV.
  UserWarning,


----------
iter: 1
n_candidates: 4
n_resources: 1500
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 2
n_candidates: 2
n_resources: 4500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'poly_ft__interaction_only': False, 'poly_ft__include_bias': False, 'poly_ft__degree': 1}
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1, include_bias=False)),
                ('lin_reg', LinearRegression())])


Best Model:
```
PolynomialFeatures(degree=1, include_bias=False)
```


In [29]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1, include_bias=False)),
    ('lin_reg'  ,   LinearRegression())
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=1, include_bias=False)),
                ('lin_reg', LinearRegression())])

In [30]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.587,RMSE=-63.494
(CV) R2=0.543,RMSE=-65.175


In [31]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.900,RMSE=-57.057


No clear benefit from polynomial features in linear RUL for simple linear regression. Actually, the degree stayed in 1, i.e., without those features.

## PolyFeatures + Non-Linear RUL

In [32]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [34]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1)
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 107}, 'poly_ft__interaction_only': True, 'poly_ft__include_bias': False, 'poly_ft__degree': 1}
Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=True)),
                ('trf_reg',
                 TransformedTargetRegressor(che

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 107}, 
PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=True))
```


In [36]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=True)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearRegression(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':107})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=True)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearRegression(),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f4790d58c20>,
                                                                            kw_args={'a_max': 107,
                                                                                     'a_min': 0})))])

In [37]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.780,RMSE=-15.763
(CV) R2=0.766,RMSE=-16.087


In [38]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.753,RMSE=-17.115


There was a slight increase in performance when comparing to the scenario without the polynomial features.

# SGD Regressor

The class SGDRegressor implements a plain stochastic gradient descent learning routine which supports different loss functions and penalties to fit linear regression models. **SGDRegressor is well suited for regression problems with a large number of training samples (> 10.000)**, for other problems we recommend Ridge, Lasso, or ElasticNet.

## Linear RUL

In [39]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor())
])

In [40]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [42]:
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "lin_reg__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "lin_reg__penalty": ['l2', 'l1', 'elasticnet'],
        "lin_reg__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "lin_reg__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__max_iter": [1000,1500,2000,2500,3000],
        "lin_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__learning_rate":['invscaling','constant','optimal','adaptive'],
        "lin_reg__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "lin_reg__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "lin_reg__early_stopping": [True,False],
        "lin_reg__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "lin_reg__n_iter_no_change": [1,2,5,8,10,15,20],
        "lin_reg__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'lin_reg__validation_fraction': 0.1, 'lin_reg__tol': 1e-07, 'lin_reg__power_t': 0.5, 'lin_reg__penalty': 'l1', 'lin_reg__n_iter_no_change': 1, 'lin_reg__max_iter': 3000, 'lin_reg__loss': 'squared_error', 'lin_reg__learning_rate': 'invscaling', 'lin_reg__eta0': 0.01, 'lin_reg__epsilon': 1.1, 'lin_reg__early_stopping': False, 'lin_reg__average': 20, 'lin_reg__alpha': 0.0001}
Pipe

Best Model:
```
SGDRegressor(average=20, epsilon=1.1, max_iter=3000,
            n_iter_no_change=1, penalty='l1', power_t=0.5,
            tol=1e-07))
```

In [43]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor(average=20, epsilon=1.1, max_iter=3000,
                        n_iter_no_change=1, penalty='l1', power_t=0.5,
                        tol=1e-07))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(average=20, epsilon=1.1, max_iter=3000,
                              n_iter_no_change=1, penalty='l1', power_t=0.5,
                              tol=1e-07))])

In [44]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.586,RMSE=-63.567
(CV) R2=0.544,RMSE=-64.895


In [45]:
X_test = test_last
eval.show_result(y_test, model.predict(X_test))

R2=-0.792,RMSE=-55.419


Poor results without non-lineariy of RUL. Comparable to regular LinearRegressor.

## Non-linear RUL

In [46]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = SGDRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [47]:
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "trf_reg__regressor__penalty": ['l2', 'l1', 'elasticnet'],
        "trf_reg__regressor__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "trf_reg__regressor__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__max_iter": [1000,1500,2000,2500,3000],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__learning_rate":['invscaling','constant','optimal','adaptive'],
        "trf_reg__regressor__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "trf_reg__regressor__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "trf_reg__regressor__early_stopping": [True,False],
        "trf_reg__regressor__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "trf_reg__regressor__n_iter_no_change": [1,2,5,8,10,15,20],
        "trf_reg__regressor__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 107}, 'trf_reg__regressor__validation_fraction': 0.6, 'trf_reg__regressor__tol': 0.001, 'trf_reg__regressor__power_t': 0.1, 'trf_reg__regressor__penalty': 'l1', 'trf_reg__regressor__n_iter_no_change': 10, 'trf_reg__regressor__max_iter': 1000, 'trf_reg__regressor__loss': 'squared_error', 'trf_reg__regressor__learning_rate': 

Best Model:
```
kw_args={'a_max': 107,'a_min': 0}))

SGDRegressor(epsilon=6,
            eta0=0.0001,
            learning_rate='constant',
            n_iter_no_change=10,
            penalty='l1',
            power_t=0.1,
            validation_fraction=0.6),
```

In [48]:
model = Pipeline([
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor=SGDRegressor(epsilon=6,
                                eta0=0.0001,
                                learning_rate='constant',
                                n_iter_no_change=10,
                                penalty='l1',
                                power_t=0.1,
                                validation_fraction=0.6),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':107})))
    ])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=SGDRegressor(epsilon=6,
                                                                   eta0=0.0001,
                                                                   learning_rate='constant',
                                                                   n_iter_no_change=10,
                                                                   penalty='l1',
                                                                   power_t=0.1,
                                                                   validation_fraction=0.6),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f4790d58c20>,
                                                                            kw_args={'a_max': 107,
                                                     

In [49]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.780,RMSE=-15.767
(CV) R2=0.767,RMSE=-16.082


In [50]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.755,RMSE=-17.036


Achieved training results from non-linear RUL Linear Regressor, but with a little better test result. However the LinearRegressor had a clipping of 121 instead of 104.

## PolyFeatures + Linear RUL

In [51]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('lin_reg'  ,   SGDRegressor())
])

In [52]:
# ~5min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "lin_reg__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "lin_reg__penalty": ['l2', 'l1', 'elasticnet'],
        "lin_reg__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "lin_reg__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__max_iter": [1000,1500,2000,2500,3000],
        "lin_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "lin_reg__learning_rate":['invscaling','constant','optimal','adaptive'],
        "lin_reg__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "lin_reg__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "lin_reg__early_stopping": [True,False],
        "lin_reg__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "lin_reg__n_iter_no_change": [1,2,5,8,10,15,20],
        "lin_reg__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions,
                            ignore_warnings=True, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'poly_ft__interaction_only': False, 'poly_ft__include_bias': False, 'poly_ft__degree': 3, 'lin_reg__validation_fraction': 0.1, 'lin_reg__tol': 0.0001, 'lin_reg__power_t': 0.25, 'lin_reg__penalty': 'elasticnet', 'lin_reg__n_iter_no_change': 10, 'lin_reg__max_iter': 2000, 'lin_reg__loss': 'squared_error', 'lin_reg__learning_rate': 'adaptive', 'lin_reg__eta0': 0.0001, 'lin_reg__ep

Best Model:
```
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(alpha=1e-06, average=1, epsilon=6, eta0=0.0001,
                              learning_rate='adaptive', max_iter=2000,
                              n_iter_no_change=10, penalty='elasticnet',
                              tol=0.0001)
```


In [53]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
    ('scaler', StandardScaler()),
    ('lin_reg',  SGDRegressor(alpha=1e-06, average=1, epsilon=6, eta0=0.0001,
                              learning_rate='adaptive', max_iter=2000,
                              n_iter_no_change=10, penalty='elasticnet',
                              tol=0.0001))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3, include_bias=False)),
                ('scaler', StandardScaler()),
                ('lin_reg',
                 SGDRegressor(alpha=1e-06, average=1, epsilon=6, eta0=0.0001,
                              learning_rate='adaptive', max_iter=2000,
                              n_iter_no_change=10, penalty='elasticnet',
                              tol=0.0001))])

In [54]:
# 10min
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.612,RMSE=-61.593
(CV) R2=0.561,RMSE=-63.814


In [55]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.776,RMSE=-55.173


No clear benefit from polynomial features in linear RUL as well. It just performs a little better than the LinearRegression model. It's the exactly same model as in FD001.

## PolyFeatures + Non-Linear RUL

In [56]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = SGDRegressor(),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [57]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__loss": ['squared_error', 'huber', 'epsilon_insensitive'],
        "trf_reg__regressor__penalty": ['l2', 'l1', 'elasticnet'],
        "trf_reg__regressor__epsilon": [1.1,1.35,1.5,2,2.5,3,3.5,4,4.5,5.5,6],
        "trf_reg__regressor__alpha":[1e-2,1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__max_iter": [1000,1500,2000,2500,3000,4000,4500],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7],
        "trf_reg__regressor__learning_rate":['invscaling','constant','optimal','adaptive'],
        "trf_reg__regressor__eta0": [1e-1,1e-2,1e-3,1e-4,1e-5,1e-6],
        "trf_reg__regressor__power_t": [1.25,1,0.75,0.5,0.25,0.1],
        "trf_reg__regressor__early_stopping": [True,False],
        "trf_reg__regressor__validation_fraction": [0.1,0.2,0.3,0.4,0.5,0.6],
        "trf_reg__regressor__n_iter_no_change": [1,2,5,8,10,15,20],
        "trf_reg__regressor__average": [False, 1,2,5,10,20]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                             param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 115}, 'trf_reg__regressor__validation_fraction': 0.5, 'trf_reg__regressor__tol': 1e-06, 'trf_reg__regressor__power_t': 0.25, 'trf_reg__regressor__penalty': 'elasticnet', 'trf_reg__regressor__n_iter_no_change': 15, 'trf_reg__regressor__max_iter': 2500, 'trf_reg__regressor__loss': 'squared_error', 'trf_reg__regressor__learnin

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 115}, 
PolynomialFeatures(include_bias=False, interaction_only=True)),
SGDRegressor(alpha=1e-05,
            average=20,
            early_stopping=True,
            epsilon=1.5,
            max_iter=2500,
            n_iter_no_change=15,
            penalty='elasticnet',
            tol=1e-06,
            validation_fraction=0.5),
```


In [58]:
model = Pipeline([
    ('poly_ft',PolynomialFeatures(include_bias=False, interaction_only=True)),
    ('scaler', StandardScaler()),
    ('trf_reg',
        TransformedTargetRegressor(
            check_inverse=False,
            regressor=SGDRegressor(alpha=1e-05,
                                    average=20,
                                    early_stopping=True,
                                    epsilon=1.5,
                                    max_iter=2500,
                                    n_iter_no_change=15,
                                    penalty='elasticnet',
                                    tol=1e-06,
                                    validation_fraction=0.5),
            transformer=FunctionTransformer(np.clip,
                                            kw_args={'a_max': 115,
                                                     'a_min': 0})))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=SGDRegressor(alpha=1e-05,
                                                                   average=20,
                                                                   early_stopping=True,
                                                                   epsilon=1.5,
                                                                   max_iter=2500,
                                                                   n_iter_no_change=15,
                                                                   penalty='elasticnet',
                                                                   tol=1e-06,
                                                                   validation_fraction=0

In [59]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.779,RMSE=-17.284
(CV) R2=0.763,RMSE=-17.677


In [60]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.733,RMSE=-19.049


With 2-degree terms, we increased the test performance. The training is similar to the previous cases, a little smaller, probably due to the random search.

# SVR

We'll use only the linear kernel, because the algorithm is quadratic on the number of samples. Otherwise, the training times get worser.

## Linear RUL

In [61]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(random_state=42))
])

In [62]:
# We need specific shape 1D arrays for this model
y_train = np.array(y_train).ravel()

In [63]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "svm_reg__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "svm_reg__fit_intercept": [True, False],
        "svm_reg__max_iter": [1000,3000,5000],
        "svm_reg__epsilon": [1,1e-1,1e-2,1e-3],
        "svm_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "svm_reg__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'svm_reg__tol': 0.001, 'svm_reg__max_iter': 5000, 'svm_reg__loss': 'squared_epsilon_insensitive', 'svm_reg__fit_intercept': True, 'svm_reg__epsilon': 0.001, 'svm_reg__C': 1, 'scaler': MinMaxScaler()}
Pipeline(steps=[('scaler', MinMaxScaler()),
                ('svm_reg',
                 LinearSVR(C=1, epsilon=0.001,
                           loss='squared_epsilon_insensitive'

Best Model:
```
('scaler', MinMaxScaler()),
LinearSVR(C=1, epsilon=0.001,
        loss='squared_epsilon_insensitive', max_iter=5000,
        random_state=42, tol=0.001))
```

In [64]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(C=1, epsilon=0.001,
        loss='squared_epsilon_insensitive', max_iter=5000,
        random_state=42, tol=0.001))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('svm_reg',
                 LinearSVR(C=1, epsilon=0.001,
                           loss='squared_epsilon_insensitive', max_iter=5000,
                           random_state=42, tol=0.001))])

In [65]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.587,RMSE=-63.495
(CV) R2=0.543,RMSE=-65.172


In [66]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.899,RMSE=-57.039


Poor results without non-lineariy of RUL. Comparable to regular LinearRegressor.

## Non-linear RUL

In [67]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearSVR(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [68]:
# ~4min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "trf_reg__regressor__fit_intercept": [True, False],
        "trf_reg__regressor__max_iter": [1000,3000,5000],
        "trf_reg__regressor__epsilon": [1,1e-1,1e-2,1e-3],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "trf_reg__regressor__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 111}, 'trf_reg__regressor__tol': 1e-05, 'trf_reg__regressor__max_iter': 5000, 'trf_reg__regressor__loss': 'squared_epsilon_insensitive', 'trf_reg__regressor__fit_intercept': True, 'trf_reg__regressor__epsilon': 0.001, 'trf_reg__regressor__C': 2, 'scaler': MinMaxScaler()}
Pipeline(steps=[('scaler', MinMaxScaler()),
         

Best Model:
```
kw_args={'a_max': 111,'a_min': 0}))
('scaler', MinMaxScaler())
LinearSVR(C=2,
            epsilon=0.001,
            loss='squared_epsilon_insensitive',
            max_iter=5000,
            random_state=42,
            tol=1e-05),
```

In [69]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor=LinearSVR(C=2,
                            epsilon=0.001,
                            loss='squared_epsilon_insensitive',
                            max_iter=5000,
                            random_state=42,
                            tol=1e-05),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':111})))
    ])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearSVR(C=2,
                                                                epsilon=0.001,
                                                                loss='squared_epsilon_insensitive',
                                                                max_iter=5000,
                                                                random_state=42,
                                                                tol=1e-05),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f4790d58c20>,
                                                                            kw_args={'a_max': 111,
                                                                                     'a_min': 0})))])

In [70]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.779,RMSE=-16.518
(CV) R2=0.765,RMSE=-16.854


In [71]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.744,RMSE=-18.062


Good linear results

## PolyFeatures + Linear RUL

In [72]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler'   ,   MinMaxScaler()),
    ('svm_reg'  ,   LinearSVR(random_state=42))
])

In [73]:
# ~20min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "svm_reg__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "svm_reg__fit_intercept": [True, False],
        "svm_reg__max_iter": [1000,3000,5000],
        "svm_reg__epsilon": [1,1e-1,1e-2,1e-3],
        "svm_reg__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "svm_reg__C":[1,2,5,10]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions,
                            ignore_warnings=True, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 24720
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 49
n_resources: 500
Fitting 5 folds for each of 49 candidates, totalling 245 fits
----------
iter: 1
n_candidates: 17
n_resources: 1500
Fitting 5 folds for each of 17 candidates, totalling 85 fits
----------
iter: 2
n_candidates: 6
n_resources: 4500
Fitting 5 folds for each of 6 candidates, totalling 30 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'svm_reg__tol': 0.001, 'svm_reg__max_iter': 5000, 'svm_reg__loss': 'squared_epsilon_insensitive', 'svm_reg__fit_intercept': True, 'svm_reg__epsilon': 0.01, 'svm_reg__C': 10, 'scaler': StandardScaler(), 'poly_ft__interaction_only': True, 'poly_ft__include_bias': False, 'poly_ft__degree': 3}
Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=Fa

Best Model:
```
 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
('scaler', StandardScaler()),
LinearSVR(C=10, epsilon=0.01,
            loss='squared_epsilon_insensitive', max_iter=5000,
            random_state=42, tol=0.001))
```


In [74]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler', StandardScaler()),
    ('svm_reg', LinearSVR(C=10, epsilon=0.01,
            loss='squared_epsilon_insensitive', max_iter=5000,
            random_state=42, tol=0.001))
])
model

Pipeline(steps=[('poly_ft',
                 PolynomialFeatures(degree=3, include_bias=False,
                                    interaction_only=True)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('scaler', StandardScaler()),
                ('svm_reg',
                 LinearSVR(C=10, epsilon=0.01,
                           loss='squared_epsilon_insensitive', max_iter=5000,
                           random_state=42, tol=0.001))])

In [75]:
# ~20min
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=0.630,RMSE=-60.135


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number

(CV) R2=0.571,RMSE=-62.946


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


In [76]:
eval.show_result(y_test, model.predict(X_test))

R2=-0.685,RMSE=-53.734


Minor benefit from polynomial features in linear RUL.  However, most of the time, it didn't converge, even with 5k iterations.

## PolyFeatures + Non-Linear RUL

In [77]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('selection', SelectFromModel(DecisionTreeRegressor())),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = LinearSVR(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
# ~15min
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [StandardScaler(), MinMaxScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__loss": ["epsilon_insensitive","squared_epsilon_insensitive"],
        "trf_reg__regressor__fit_intercept": [True, False],
        "trf_reg__regressor__max_iter": [1000,3000,5000],
        "trf_reg__regressor__epsilon": [1,1e-1,1e-2,1e-3],
        "trf_reg__regressor__tol": [1e-3,1e-4,1e-5,1e-6,1e-7,1e-8],
        "trf_reg__regressor__C":[1,2,5,10],
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                             param_distributions, ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 93}, 'trf_reg__regressor__tol': 0.001, 'trf_reg__regressor__max_iter': 3000, 'trf_reg__regressor__loss': 'epsilon_insensitive', 'trf_reg__regressor__fit_intercept': True, 'trf_reg__regressor__epsilon': 1, 'trf_reg__regressor__C': 10, 'scaler': StandardScaler(), 'poly_ft__interaction_only': False, 'poly_ft__include_bias': Tr

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 93}, 
PolynomialFeatures(degree=3)
('scaler', StandardScaler()),
LinearSVR(C=10, epsilon=1,
        max_iter=3000,
        random_state=42,
        tol=0.001),
```


In [ ]:
model = Pipeline([
    ('poly_ft',PolynomialFeatures(degree=3)),
    ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
    ('scaler', StandardScaler()),
    ('trf_reg',
        TransformedTargetRegressor(
            check_inverse=False,
            regressor=LinearSVR(C=10, epsilon=1,
                                max_iter=3000,
                                random_state=42,
                                tol=0.001),
            transformer=FunctionTransformer(np.clip,
                                            kw_args={'a_max': 93,
                                                     'a_min': 0})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('selection',
                 SelectFromModel(estimator=DecisionTreeRegressor())),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=LinearSVR(C=10, epsilon=1,
                                                                max_iter=3000,
                                                                random_state=42,
                                                                tol=0.001),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f48ef421cb0>,
                                                                            kw_args={'a_max': 93,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:1208: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  ConvergenceWarning,


R2=0.806,RMSE=-13.010
(CV) R2=0.799,RMSE=-13.185


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.789,RMSE=-14.641


There was a case of failing to converge. But we achieved the best results in testing using linear kernel models.